# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd

csv_file_path = 'C:\\Users\\David Santos\\Desktop\\training.1600000.processed.noemoticon.csv'
df = pd.read_csv(csv_file_path, encoding='latin-1')


In [2]:
sampled_df = df.sample(n=5000, random_state=42)

sampled_df.shape[0]

5000

In [3]:
sampled_df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick"


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [4]:
import pandas as pd
sampled_df.rename(columns={'0': 'target',
                           '1467810369': 'id',
                           'Mon Apr 06 22:19:45 PDT 2009': 'date',
                           'NO_QUERY': 'flag',
                           '_TheSpecialOne_': 'user',
                           '@switchfoot http://twitpic.com/2y1zl - Awww, that\'s a bummer. You shoulda got David Carr of Third Day to do it. ;D.': 'text'}, inplace=True)


In [5]:
problematic_column_name = sampled_df.columns[5] # actual index of the column you want to rename


sampled_df.rename(columns={problematic_column_name: 'text'}, inplace=True)

In [6]:
sampled_df.head()

,target,id,date,flag,user,text
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick"


In [7]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\David
[nltk_data]     Santos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\David
[nltk_data]     Santos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\David
[nltk_data]     Santos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\David
[nltk_data]     Santos\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
def clean_up(text):
  
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize(text):
  
    tokens = nltk.word_tokenize(text)
    return tokens

def stem_and_lemmatize(tokens):
   
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    processed_tokens = [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]
    return processed_tokens

def remove_stopwords(tokens):

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens


sampled_df['text_processed'] = sampled_df['text'].apply(clean_up)
sampled_df['text_processed'] = sampled_df['text_processed'].apply(tokenize)
sampled_df['text_processed'] = sampled_df['text_processed'].apply(stem_and_lemmatize)
sampled_df['text_processed'] = sampled_df['text_processed'].apply(remove_stopwords)

sampled_df.head()

,target,id,date,flag,user,text,text_processed
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,"[nkluvreva, poor, littl, dumpl, holmdel, vid, ..."
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,"[im, bed, got, ta, wake, hella, earli, tomorro..."
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,"[havent, abl, listen, yet, speaker, bust]"
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,"[rememb, whi, solv, rel, big, equat, two, unkn..."
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick","[ate, much, feel, sick]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
import nltk
from nltk.probability import FreqDist


all_words = [word for sublist in sampled_df['text_processed'] for word in sublist]


word_freq = FreqDist(all_words)


top_words = word_freq.most_common(5000)


top_5000_words = [word[0] for word in top_words]

print(top_5000_words)

['im', 'go', 'get', 'wa', 'day', 'work', 'thi', 'good', 'like', 'love', 'got', 'cant', 'dont', 'know', 'today', 'time', 'want', 'miss', 'thank', 'one', 'u', 'feel', 'lol', 'back', 'make', 'see', 'well', 'realli', 'think', 'new', 'still', 'ha', 'look', 'need', 'oh', 'na', 'night', 'watch', 'tomorrow', 'much', 'follow', 'wish', 'last', 'wait', 'morn', 'amp', 'hope', 'home', 'come', 'great', 'ill', 'tri', 'fun', 'whi', 'didnt', 'bad', 'though', 'sleep', 'haha', 'friend', 'twitter', 'nice', 'sorri', 'happi', 'veri', 'sad', 'right', 'start', 'hate', 'gon', 'say', 'would', 'take', 'week', 'thing', 'show', 'hi', 'peopl', 'final', 'everyon', 'onli', 'even', 'plea', 'better', 'awesom', 'tonight', 'let', 'school', 'weekend', 'sure', 'ive', 'way', 'tweet', 'could', 'alway', 'use', 'littl', 'bed', 'hey', 'someth', 'life', 'hour', 'play', 'suck', 'girl', 'alreadi', 'omg', 'yeah', 'man', 'movi', 'next', 'tell', 'soon', 'anoth', 'game', 'never', 'ye', 'guy', 'year', 'rain', 'sick', 'ok', 'guess', 'x'

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [10]:
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize


all_words = [word for sublist in sampled_df['text_processed'] for word in sublist]


word_freq = FreqDist(all_words)


top_words = word_freq.most_common(5000)


top_5000_words = [word[0] for word in top_words]




tweets = ['This is a positive tweet.', 'Negative sentiment in this one.', 'Another positive example.', 'No negativity here.']
sentiments = ['positive', 'negative', 'positive', 'negative']


feature_set = []


for tweet, sentiment in zip(tweets, sentiments):
  
    words = word_tokenize(tweet)
    
   
    document_features = {word: (word in words) for word in top_5000_words}
    
    feature_set.append((document_features, sentiment == 'positive'))


#acho que está mal


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [11]:
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize



all_words = [word for sublist in sampled_df['text_processed'] for word in sublist]


word_freq = FreqDist(all_words)


top_words = word_freq.most_common(5000)


top_5000_words = [word[0] for word in top_words]



tweets = ['This is a positive tweet.', 'Negative sentiment in this one.', 'Another positive example.', 'No negativity here.']
sentiments = ['positive', 'negative', 'positive', 'negative']


feature_set = []

for tweet, sentiment in zip(tweets, sentiments):
    words = word_tokenize(tweet)
    document_features = {word: (word in words) for word in top_5000_words}
    feature_set.append((document_features, sentiment == 'positive'))


train_size = int(0.8 * len(feature_set))
train_set = feature_set[:train_size]
test_set = feature_set[train_size:]


classifier = NaiveBayesClassifier.train(train_set)


classifier.show_most_informative_features(10)

Most Informative Features
                   tweet = False           False : True   =      1.5 : 1.0
                      aa = False           False : True   =      1.0 : 1.0
                 aaaawww = False           False : True   =      1.0 : 1.0
                    aaah = False           False : True   =      1.0 : 1.0
              aaarghwhat = False           False : True   =      1.0 : 1.0
                    aagw = False           False : True   =      1.0 : 1.0
                     aah = False           False : True   =      1.0 : 1.0
                    aane = False           False : True   =      1.0 : 1.0
                   aaron = False           False : True   =      1.0 : 1.0
                  aawwww = False           False : True   =      1.0 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here